# Real-Time Wildlife Alert System: Example Tutorial Notebook
Thomas Ratsakatika | AI and Environment Researcher | University of Cambridge

Last updated: 26 June 2024

## Introduction

This notebook will walk you through how to set up and use the advanced version of the wildlife alert system.

**Steps 1 and 2** let you experiment with the alert system using photos stored on your computer.

**Steps 3 and 4** walk you through setting up the system to receive emails from a 4G-enabled camera trap.

If you have any queries or feedback, please contact [Tom Ratsakatika](mailto:trr26@cam.ac.uk)

## Step 1: Import Modules and AI Models

First, ensure you have cloned (copied) the repository to your device, created a virtual environment and installed the required modules by following the instructions in the [README](../README.md#-example-tutorial).

You should then be able to run the cell below, which imports the modules required to run the alert system. 

In [ ]:
### Import required modules ###

from megadetector.detection import run_detector  # MegaDetector object detection model
import pandas as pd             # Data manipulation and analysis
import yaml                     # Handles YAML (config) files
import time                     # Time functions
from datetime import datetime   # Date and time manipulation
import schedule                 # Schedule function for weekly reports
import functools                # Function for weekly reports
import sys                      # System parameters and functions
sys.path.append('../scripts')   # Add scripts directory to system path

### Import modules required for the the example ###

from IPython.display import display, HTML  # Function to display images in Jupyter notebooks
import tkinter as tk            # Tkinter GUI toolkit
from tkinter import filedialog  # File dialog for opening files
from PIL import Image           # Python Imaging Library (PIL)
import os                       # Functions for interacting with the operating system
from io import BytesIO          # Handle binary data in memory
import base64                   # Encode binary data for HTML display

### Import alert system functions from ../scripts/alert_system_utils.py ###

from alert_system_utils import (

    ### Functions to download photos and metadata from emails ###

    current_time,               # Get the current time
    check_emails,               # Checks for new emails, extracts photos and metadata
    extract_and_update_camera_info,  # Extract and update camera information
    update_camera_data_dataframe,    # Update camera data DataFrame

    ### Functions to detect and classify animals in photos ###

    set_device,                 # Sets computation device (CPU/GPU)
    detector,                   # Animal/human/vehicle detection
    classifier,                 # Animal classification model
    batch_classification,       # Batch classification of images
    detections_in_sequence,     # Checks if anything has been detected

    ### Functions to annotate photos and send an alert to Telegram ###

    generate_alert_caption,     # Generate captions for alerts
    send_alert_to_telegram,     # Send alerts to Telegram
    annotate_images,            # Annotate images with detection results

    ### Functions to save the photos and send weekly reports ###
    save_images,                # Save images to disk
    send_weekly_report          # Send a weekly report
)

Now you can initialise the detection and classification models.

The alert system follows a two-stage detection and classification process. The detection model specalises in detecting animals, humans and vehicles in photos, and determining their location with bounding boxes. The classification model specialises in determining the species of animals.

This notebook uses MegaDetector for detection and DeepFaune for classification. You can change the path for the classifcation model to one of the fine-tuned [Carpathian Mountains models](../README.md#-models) if required. Changing the paths to any other detection of classification models will require more complex changes to the detector and classifier functions in the [alert system utils](../scripts/alert_system_utils.py). 

Run the cell below to initialise the detection and classifiaction models. The code will automatically use a NVIDIA GPU if you have one.

In [ ]:
# Detection Model Settings
DETECTOR_MODEL_PATH = '../models/md_v5a.0.0.pt'
DETECTOR_CLASSES = ["animal", "human", "vehicle"]

# Classification Model Settings
BACKBONE = 'vit_large_patch14_dinov2'
CLASSIFIER_MODEL_PATH = '../models/deepfaune-vit_large_patch14_dinov2.lvd142m.pt'   # Change to fine-tuned model if desired
CLASSIFIER_CLASSES = [
    "Badger", "Ibex", "Red Deer", "Chamois", "Cat",
    "Goat", "Roe Deer", "Dog", "Squirrel", "Equid", "Genet",
    "Hedgehog", "Lagomorph", "Wolf", "Lynx", "Marmot",
    "Micromammal", "Mouflon", "Sheep", "Mustelid", "Bird",
    "Bear", "Nutria", "Fox", "Wild Boar", "Cow"
]
ROMANIAN_CLASSES = [
    "Bursuc", "Ibex", "Cerb", "Capră Neagră", "Pisică", 
    "Capră", "Căprioară", "Câine", "Veveriță", "Cal", "Genetă",
    "Arici", "Iepuri", "Lup", "Râs", "Marmotă", 
    "Micromamifer", "Muflon", "Oaie", "Mustelid", "Pasăre", 
    "Urs", "Nutrie", "Vulpe", "Mistreț", "Vacă"
]
SPECIES_OF_INTEREST = ["Wild Boar", "Bear"] # Species for which priority alerts are sent (currently only supports wild boar and bears)

# Locations of capture database, camera location tables, and storage folder for photos received by the alert system
CAPTURE_DATABASE_PATH = '../data/capture_database.csv' 
CAMERA_LOCATIONS_PATH = '../data/camera_locations.csv'
PHOTOS_PATH = '../data/photos/'

# Initialise the Detection and Classifier Models
device = set_device()
detector_model = run_detector.load_detector(DETECTOR_MODEL_PATH)
print("Loading classifier...")
start_time = time.time()
classifier_model = classifier(CLASSIFIER_MODEL_PATH, BACKBONE, CLASSIFIER_CLASSES, device)
end_time = time.time()
print(f"Loaded classifier in {(end_time - start_time):.2f} seconds")

## Step 2: Test Your Setup on Example Photos

Before testing the system on some photos, we need to set the models' thresholds. These can be set between 0 and 1.

A lower **detection** threshold will mean the system is more sensitive, but it might think that objects such as log or a tree stump are animals when they are not. A higher detection threshold will mean that you get fewer false positives, but the system may miss difficult to detect animals (e.g. nighttime or blurry images).

A lower **classification** threshold will mean that the system will be more likely to try and classify an animal detection, even if it isn't sure. A higher classification threshold will mean that the model will label more animal detections as "unknown".

We also need to set the alert system language (currently English or Romanian), and the times during which we want to receive photos of people or vehicles. The latter is a privacy feature, to prevent the system from sending photos of the general public during daylight hours.

In [ ]:
DETECTION_THRESHOLD = 0.15      # Detection threshold - recommended 0.15
CLASSIFICATION_THRESHOLD = 0.20 # Classification threshold - recommend set between 0.2 to 0.8 depending on tolerance for false positives

# Alert Message Settings
ALERT_LANGUAGE = "en"           # Curently English (en) and Romanian (ro) are supported
HUMAN_ALERT_START = "21:00"     # Privacy feature: start/end time that photos of people may be sent
HUMAN_ALERT_END = "06:00"

Now you can run the cell below to run the system on a photo(s) on your device. Note how the first detects the number of objects, and then itterates through each detection in each photo using the classifier.

You can select more than one photo by holding down CTRL and click on multiple files. This mimics how the alert system may receive photos from a 4G camera trap in the field. *Note that the system does not expect to see bears and wild boars together, so if you select both, it will only count the first species it sees in the alert message header.*

In [ ]:
# Function to open file dialog and select one or more JPG images
def open_images():
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    root.geometry("800x600")  
    initial_dir = '../data/example_photos'  # Set initial directory for file dialog
    # Open file dialog to select one or more JPG files
    file_paths = filedialog.askopenfilenames(initialdir=initial_dir, title="Select file(s)",
                                             filetypes=[("JPEG files", "*.jpg"), 
                                                        ("JPEG files", "*.jpeg"), 
                                                        ("JPEG files", "*.JPG"), 
                                                        ("JPEG files", "*.JPEG")])
    # Open and return the selected images as a list of PIL Image objects
    return [Image.open(file_path) for file_path in file_paths] if file_paths else []


# Call the function and store the selected images in a list
images = open_images()

# Print the number of selected images
print(f"{current_time()} | {len(images)} image(s) selected")

# Load the capture database into a DataFrame, df
# This DataFrame is required to manage data in the subsequent steps
df = pd.read_csv(CAPTURE_DATABASE_PATH)

# Loop to add dummy rows to the DataFrame for this example
for _ in range(len(images)):
    new_row = pd.DataFrame([['Example', 1] + ['Example'] * 10 + [None] * (len(df.columns) - 12)], columns=df.columns)
    df = pd.concat([df, new_row], ignore_index=True)

# Detect animals, humans and/or vehicles in the image(s) using the MegaDetector detection model
df, human_warning = detector(df, detector_model, images, DETECTION_THRESHOLD)

# Classify objects in the image(s) using the DeepFaune classification model
df = batch_classification(df, classifier_model, images, CLASSIFICATION_THRESHOLD)

# Check if there are any detections in the sequence above the threshold
if detections_in_sequence(df, images):
    # Generate an alert caption based on the detections
    df, alert_caption = generate_alert_caption(df, human_warning, HUMAN_ALERT_START, HUMAN_ALERT_END, len(images), SPECIES_OF_INTEREST, "example@email.com", ALERT_LANGUAGE, CLASSIFIER_CLASSES, ROMANIAN_CLASSES)
    human_warning = False  # Reset human warning flag
    # Annotate images with bounding boxes, labels and confidence levels
    images = annotate_images(df, images, human_warning, HUMAN_ALERT_START, HUMAN_ALERT_END, ALERT_LANGUAGE, CLASSIFIER_CLASSES, ROMANIAN_CLASSES)

# Display the images in Jupyter notebook
if images:
    html = "<div style='display: flex; flex-wrap: wrap;'>"
    for img in images:
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        img_str = base64.b64encode(buffered.getvalue()).decode()
        html += f"<div style='margin: 10px;'><img src='data:image/jpeg;base64,{img_str}' width='600px' /></div>"
    html += "</div>"
    display(HTML(html))

# Display the alert message in Jupyter notebook
alert_html = f"<div style='padding: 10px; background-color: white; border: 1px solid black; white-space: pre-wrap; color: black; width: 600px;'>{alert_caption}</div>"
display(HTML(alert_html))


## Step 3: Set Up an Alert System Email Account and Telegram Group


### Email Setup

To set up the system so that it can receive emails from a 4G camera trap, you need to create a dedicated email account to receive the camera trap images. Your email provider **must** support app passwords (e.g. <a href="https://myaccount.google.com/apppasswords" target="_blank">Gmail</a>). Create an app password and keep a record of it. Then set up your 4G camera trap to send photos to this dedicated email address.

### Telegram Setup

Setup Telegram on your mobile device. Create a bot in Telegram using @BotFather and note down the bot token. Detailed instructions on how to do this can be <a href="https://core.telegram.org/bots/tutorial" target="_blank">found here</a>.

Create a group in Telegram, add the bot to the group and make it an admin. Then note down the group's <a href="https://www.wikihow.com/Know-Chat-ID-on-Telegram-on-Android" target="_blank">chat ID</a> (do this AFTER adding the bot - the chat ID should start with '#-100')

### Update the Config.yaml file

Open the [config.yaml](../config.yaml) file in a text editor. It should be in this format.

```
imap_config:
  host: 'YOUR_IMAP_SERVER_NAME'
  user: 'YOUR_DEDICATED_EMAIL_ADDRESS'
  password: 'YOUR_APP_PASSWORD'

telegram_config:
  bot_token: 'YOUR_BOT_TOKEN'
  chat_id: 'YOUR_CHAT_ID'

smtp_config:
  host: 'YOUR_SMTP_SERVER_NAME'
  port: 'YOUR_SMTP_SERVER_PORT'
  user: 'YOUR_DEDICATED_EMAIL_ADDRESS'
  password: 'YOUR_APP_PASSWORD'
  recipients:
    - 'RECIPIENT_EMAIL_ADDRESS_1'
    - 'RECIPIENT_EMAIL_ADDRESS_2'
```

Update the settings with your email providers imap and smtp settings, and your Telegram bot token and chat ID.

The 'recipients' under smtp_config are the email addresses that will receive a regular report. You can change when and how often the regular report is sent in the next cell.

**DO NOT share your config.yaml file with anyone as it contains passwords necessary to access your dedicated email account.**

### Update the camera(s) details

Update the [camera locations](data/camera_locations.csv) CSV file with your camera(s) details, location and a google maps link.

You can add as many cameras as you like.

### Load your configuration

Run the cell below to load your configuration.

In [ ]:

# Load settings from configuration file
with open('../config.yaml') as file:
    config = yaml.safe_load(file)

IMAP_HOST = config['imap_config']['host']
EMAIL_USER = config['imap_config']['user']
EMAIL_PASS = config['imap_config']['password']

TELEGRAM_BOT_TOKEN = config['telegram_config']['bot_token']
TELEGRAM_CHAT_ID =  '-1002249589791' # config['telegram_config']['chat_id']  #  replace with config after tests # 

SMTP_SERVER = config['smtp_config']['host']
SMTP_PORT = int(config['smtp_config']['port'])
EMAIL_SENDER = config['smtp_config']['user']
EMAIL_PASSWORD = config['smtp_config']['password']
RECIPIENTS = config['smtp_config']['recipients']

CHECK_EMAIL_FREQUENCY = 60      # Sets how often the system checks for emails (default - 60 seconds)

### Settings for regular report (default - weekly on Mondays at 08:00)
schedule.every().monday.at("08:00").do(
    functools.partial(send_weekly_report, SMTP_SERVER, EMAIL_SENDER, EMAIL_PASSWORD, SMTP_PORT, CAPTURE_DATABASE_PATH, CAMERA_LOCATIONS_PATH, RECIPIENTS, EMAIL_USER)
)

## Step 4: Test the System

You can now run the cell below to launch the alert system.

The code will check the email account every 60 seconds for unread emails, download any photos, detect and classify animals, and send an alert your Telegram Group.

It will then update the [capture database](data/capture_database.csv) and save the original photos in the [photos folder](data/photos).

A high level process flow diagramme is shown below the code.

In [ ]:
if __name__ == "__main__":
    print(f"\n{current_time()} | Monitoring {EMAIL_USER} for new messages...")
    while True:
        try:
            # Check for emails, extract metadata
            images, original_images, camera_id, temp_deg_c, img_date, img_time, battery, sd_memory = \
                check_emails(IMAP_HOST, EMAIL_USER, EMAIL_PASS)
            
            if camera_id:
                
                # Get the camera details from the camera_location databse
                camera_make, gps, location, map_url, battery, sd_memory = extract_and_update_camera_info(CAMERA_LOCATIONS_PATH, camera_id, battery, sd_memory)
                
                if images:
                    
                    # If images are attached to the email, open the capture_database
                    df = pd.read_csv(CAPTURE_DATABASE_PATH)
                    
                    # Update the capture database with rows for each image
                    df = update_camera_data_dataframe(df, len(images), camera_id, camera_make, img_date, img_time, temp_deg_c, battery, sd_memory, location, gps, map_url)
                    
                    # Run the detection model on each image
                    df, human_warning = detector(df, detector_model, images, DETECTION_THRESHOLD)
                    
                    # Run the classification model on each image
                    df = batch_classification(df, classifier_model, images, CLASSIFICATION_THRESHOLD)
                    
                    if detections_in_sequence(df, images):
                        
                        # If there are any detections above the threshold, create an alert caption
                        df, alert_caption = generate_alert_caption(df, human_warning, HUMAN_ALERT_START, HUMAN_ALERT_END, len(images), SPECIES_OF_INTEREST, EMAIL_USER, ALERT_LANGUAGE, CLASSIFIER_CLASSES, ROMANIAN_CLASSES)                        
                        
                        # If no humans/vehicles are detected, annotate the photos with boxes and species labels
                        alert_images = annotate_images(df, images, human_warning, HUMAN_ALERT_START, HUMAN_ALERT_END, ALERT_LANGUAGE, CLASSIFIER_CLASSES, ROMANIAN_CLASSES)
                        
                        # Send an alert to the Telegram group
                        send_alert_to_telegram(TELEGRAM_BOT_TOKEN, TELEGRAM_CHAT_ID, alert_images, alert_caption)
                    
                    else:
                        
                        print(f"{current_time()} | All photos in sequence are empty")
                    
                    # Save the original photos the the ../data/photos/ folder
                    df = save_images(df, original_images, human_warning, PHOTOS_PATH)
                    
                    # Update the capture_database
                    df.to_csv(CAPTURE_DATABASE_PATH, index=False)
                    print(f"{current_time()} | Capture database updated: {CAPTURE_DATABASE_PATH}")
                    
                    # Clear the dataframe to free up memory
                    del df
                
                else:
                    
                    print(f"{current_time()} | No images attached to email")
                
                print(f"\n{current_time()} | Monitoring {EMAIL_USER} for new messages...")
            
            else:
                # Wait before checking emails again
                time.sleep(CHECK_EMAIL_FREQUENCY)
            
            # Check to see if it is time to send the regular report
            schedule.run_pending()
        
        except KeyboardInterrupt:
            print(f"{current_time()} | Interrupted by user")
            break
        
        # Error handling to keep the system running if an error occurs
        except Exception as e:
            print(f"{current_time()} | An error occurred: {e}")
            time.sleep(CHECK_EMAIL_FREQUENCY)
            print(f"\n{current_time()} | Monitoring {EMAIL_USER} for new messages...")
            continue

![Flow Diagram](../assets/final_alert_system_flow_diagram.png)

## Step 5: Deploy Your Script

That's it. The code in this notebook is identical to the [advanced_alert_system.py](scripts/advanced_alert_system.py) script, which you can now edit to align with your required settings.

You need to run the [advanced_alert_system.py](../scripts/advanced_alert_system.py) script on a device that can be left on constantly. This may be an old computer or a virtual machine on your server. Consult your IT team to determine the best option.

You could alternatively run the [basic_alert_system.py](../scripts/basic_alert_system.py) script on a Raspberry Pi 4B. The logic is similar to this advanced system, however the models and messages are simpler. A Raspberry Pi 5 may even be able to run the advanced version, although this has not been tested.